- Read data
- Filter the data by too much missing values
- Aggregation pipeline
- Merge pipeline
- Train base line model

In [1]:
import pandas as pd
import polars as pl
import time

Config

In [2]:
train_path = './home-credit-credit-risk-model-stability/csv_files/train/'
test_path = './home-credit-credit-risk-model-stability/csv_files/test/'
result_path = './home-credit-credit-risk-model-stability/csv_files/results/'

### Help functions

In [3]:
def set_dtypes(df : pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64))
        elif col[-1] in ("D"):
            df = df.with_columns(pl.col(col).cast(pl.Date))
        elif col[-1] in ("M"):
            df = df.with_columns(pl.col(col).cast(pl.String))
        elif col in ['date_decision']:
            df = df.with_columns(pl.col(col).cast(pl.Date))
            
    return df

In [4]:
# drop the column which has 95% null

def missing_value_pct(df : pl.DataFrame, base_columns : list, target_columns : list, threshold : float) -> pl.DataFrame:
    df_base = df.select(pl.col(base_columns).n_unique())
    
    df_sele = df.select(target_columns)
    df_missing = df_sele.select([pl.col(col).is_null().sum() for col in target_columns])
    
    result = pl.concat([df_base, df_missing], how='horizontal')
    
    uni_base = df.select(pl.col(base_columns)).n_unique()
    
    result = result.select([
        pl.col(col)/uni_base for col in result.columns
    ])
    
    result = result.transpose(include_header=True).rename({"column":"column_name", "column_0":"missing_pct"}) \
                    .sort(by='missing_pct', descending=True)
    
    sele_cols = result.filter(pl.col('missing_pct') < threshold).select(pl.col('column_name'))

    df_result = df.select(pl.col(base_columns + [col for col in target_columns if col in sele_cols.select(pl.col('column_name')).to_series()]))
    
    return df_result  

### Depth 0

base 

In [ ]:
train_base = pl.read_csv(train_path +'train_base.csv')
train_base.head(5)

In [ ]:
train_base.shape

In [ ]:
def statistic_cleaning():
    train_statistic_0 = pl.read_csv(train_path + 'train_static_0_0.csv').pipe(set_dtypes)
    train_statistic_1 = pl.read_csv(train_path + 'train_static_0_1.csv').pipe(set_dtypes)
    train_statistic = pl.concat([train_statistic_0, train_statistic_1], how='vertical_relaxed')
    train_statistic_miss_fil = missing_value_pct(train_statistic, ['case_id'], [col for col in train_statistic.columns if col != 'case_id'], 0.90)
    train_statistic_miss_fil.write_csv('./home-credit-credit-risk-model-stability/csv_files/results/train_statistic_miss_fil.csv')
    
def cb_cleaning():
    train_cb_0 = pl.read_csv(train_path + 'train_static_cb_0.csv')
    train_cb_miss_fil = missing_value_pct(train_cb_0, ['case_id'], [col for col in train_cb_0.columns if col != "case_id"], 0.90)
    train_cb_miss_fil.write_csv(result_path + 'train_cb_miss_fil.csv')
    
#statistic_cleaning()
#cb_cleaning()

### Depth 1

- aggregate by case_is, num_group1

applprev_1

In [ ]:
def applprev_1_cleaning():
    train_applprev_1_0 = pl.read_csv(train_path + 'train_applprev_1_0.csv').pipe(set_dtypes)
    train_applprev_1_1 = pl.read_csv(train_path + 'train_applprev_1_1.csv').pipe(set_dtypes)
    train_applprev = pl.concat([train_applprev_1_0, train_applprev_1_1], how='vertical_relaxed')
    base_columns = ['case_id', 'num_group1']
    tar_columns = [col for col in train_applprev.columns if col not in base_columns]
    train_applprev_miss_fil = missing_value_pct(train_applprev, base_columns, tar_columns, 0.90)
    train_applprev_miss_fil.write_csv(result_path + 'train_applprev_1_miss_fil.csv')

# applprev_1_cleaning()

other

In [ ]:
def other_cleaning():
    train_other = pl.read_csv(train_path + 'train_other_1.csv').pipe(set_dtypes)
    base_columns = ['case_id', 'num_group1']
    tar_columns = [col for col in train_other.columns if col not in base_columns]
    train_other_miss_fil = missing_value_pct(train_other, base_columns, tar_columns, 0.90)
    train_other_miss_fil.write_csv(result_path + 'train_other_miss_fil.csv')
    
# other_cleaning()

credit_bureau_a_1

In [ ]:
def credit_bureau_a_1_cleaning():
    train_credit_bureau_a_1_0 = pl.read_csv(train_path + 'train_credit_bureau_a_1_0.csv').pipe(set_dtypes)
    train_credit_bureau_a_1_1 = pl.read_csv(train_path + 'train_credit_bureau_a_1_1.csv').pipe(set_dtypes)
    train_credit_bureau_a_1_2 = pl.read_csv(train_path + 'train_credit_bureau_a_1_2.csv').pipe(set_dtypes)
    train_credit_bureau_a_1 = pl.concat([train_credit_bureau_a_1_0, train_credit_bureau_a_1_1, train_credit_bureau_a_1_2], how='vertical_relaxed')
    base_columns = ['case_id', 'num_group1']
    tar_columns = [col for col in train_credit_bureau_a_1.columns if col not in base_columns]
    train_credit_bureau_a_1_miss_fil = missing_value_pct(train_credit_bureau_a_1, base_columns, tar_columns, 0.90)
    train_credit_bureau_a_1_miss_fil.write_csv(result_path + 'train_credit_bureau_a_1_miss_fil.csv')

credit_bureau_a_1_cleaning()

In [5]:
train_credit_bureau_a_1_0 = pl.read_csv(train_path + 'train_credit_bureau_a_1_0.csv').pipe(set_dtypes)

In [6]:
train_credit_bureau_a_1_0

case_id,annualeffectiverate_199L,annualeffectiverate_63L,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,contractsum_5085717L,credlmt_230A,credlmt_935A,dateofcredend_289D,dateofcredend_353D,dateofcredstart_181D,dateofcredstart_739D,dateofrealrepmt_138D,debtoutstand_525A,debtoverdue_47A,description_351M,dpdmax_139P,dpdmax_757P,dpdmaxdatemonth_442T,dpdmaxdatemonth_89T,dpdmaxdateyear_596T,dpdmaxdateyear_896T,financialinstitution_382M,financialinstitution_591M,instlamount_768A,instlamount_852A,interestrate_508L,lastupdate_1112D,lastupdate_388D,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,num_group1,numberofcontrsvalue_258L,…,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax2date_1002D,overdueamountmax2date_1142D,overdueamountmax_155A,overdueamountmax_35A,overdueamountmaxdatemonth_284T,overdueamountmaxdatemonth_365T,overdueamountmaxdateyear_2T,overdueamountmaxdateyear_994T,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,purposeofcred_426M,purposeofcred_874M,refreshdate_3813885D,residualamount_488A,residualamount_856A,subjectrole_182M,subjectrole_93M,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A
i64,str,f64,str,str,str,str,str,f64,f64,date,date,date,date,date,f64,f64,str,f64,f64,str,f64,f64,str,str,str,f64,f64,str,date,date,f64,f64,f64,str,i64,f64,…,f64,str,date,date,f64,str,f64,f64,f64,f64,f64,f64,date,date,f64,f64,str,f64,f64,str,str,f64,str,str,str,str,date,f64,f64,str,str,f64,f64,f64,f64,f64,f64
388,null,null,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""",null,null,null,2023-06-20,null,null,2018-06-20,null,374419.5,0.0,"""a55475b1""",0.0,null,null,7.0,2018.0,null,"""a55475b1""","""55b002a9""",null,null,null,2019-01-24,null,7811.4463,null,null,null,0,2.0,…,0.0,null,null,null,0.0,null,null,260093.7,null,0.0,0.0,null,null,null,0.0,null,null,7.0,2018.0,null,null,30.0,null,null,"""96a8fdfe""","""a55475b1""",null,null,null,"""ab3c25cf""","""ab3c25cf""",null,268897.62,0.0,0.0,374419.5,0.0
388,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""",null,null,135806.0,2020-08-06,null,null,2018-08-06,null,null,null,"""a55475b1""",0.0,null,null,8.0,2018.0,null,"""a55475b1""","""P204_66_73""",8742.8,null,null,2019-01-11,null,8742.8,null,null,null,1,null,…,0.0,null,null,null,0.0,null,null,null,null,0.0,0.0,null,null,null,0.0,null,null,8.0,2018.0,null,null,null,null,null,"""60c73645""","""a55475b1""",null,null,114325.805,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,2,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,null,"""a55475b1""",null,null,null,null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,null,null,3,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""",2019-01-28,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null
388,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,null,null,null,null,n

tax_registry_a_1

In [ ]:
tax_registry_a_1 = pl.read_csv(train_path + 'train_tax_registry_a_1.csv').pipe(set_dtypes)

### Depth 2

- aggregate by case_is, num_group1

applprev_2

In [ ]:
def applprev_2_cleaning():
    train_applprev_2 = pl.read_csv(train_path + 'train_applprev_2.csv').pipe(set_dtypes)
    base_columns = ['case_id', 'num_group1', 'num_group2']
    tar_columns = [col for col in train_applprev_2.columns if col not in base_columns]
    train_applprev_2_miss_fil = missing_value_pct(train_applprev_2, base_columns, tar_columns, 0.90)
    train_applprev_2_agg = train_applprev_2_miss_fil.group_by(['case_id','conts_type_509L']).agg(pl.count().alias('type_count'))
    train_applprev_2_agg.write_csv(result_path + 'train_applprev_2_agg.csv')
    
applprev_2_cleaning()

person_2

In [ ]:
def person_2_cleaning():
    train_person_2 = pl.read_csv(train_path + 'train_person_2.csv').pipe(set_dtypes)
    base_columns = ['case_id', 'num_group1', 'num_group2']
    tar_columns = [col for col in train_person_2.columns if col not in base_columns]
    train_person_2_miss_fil = missing_value_pct(train_person_2, base_columns, tar_columns, 0.90)
    train_person_2_miss_fil.write_csv(result_path + 'train_person_2_miss_fil.csv')
    
person_2_cleaning()

# Aggregations

### Depth 2

person_2

In [ ]:
def get_mode_value(primary_keys, value):
    return (
        pl.col(value)
        .drop_nulls()
        .mode()
        .first()
        .over(primary_keys)
        .alias(f'mode_{value}')
    )

In [ ]:
train_person_2_miss_fil = pl.read_csv(result_path + 'train_person_2_miss_fil.csv')

columns_to_process = ['addres_district_368M', 'addres_zip_823M', 'conts_role_79M', 'empls_economicalst_849M', 'empls_employer_name_740M']
train_person_2_agg = train_base.select(pl.col(['case_id']))
for col in columns_to_process:
    temp = (train_person_2_miss_fil.with_columns([
                get_mode_value(primary_keys=['case_id'], value=col)
            ])
            .select(pl.col(['case_id',f'mode_{col}']))
            .unique())
    
    train_person_2_agg = train_person_2_agg.join(temp, on='case_id', how='left')